## Felipe Ribeiro




# Bibliotecas

In [4]:
# Instalação das Bibliotecas

!pip install transformers accelerate bitsandbytes PyPDF2 --quiet
!pip install langchain sentence-transformers faiss-cpu --quiet
!pip install langchain_community

In [5]:
from huggingface_hub import login
login(new_session=True)

In [6]:
# Importações e Carregamento do Modelo
import torch
import json
import PyPDF2
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configuração para carregar o modelo com quantização de 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Nome do modelo no Hugging Face
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Carrega o modelo com a configuração de quantização
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto", # Mapeia o modelo automaticamente para a GPU
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import re

def limpar_texto(texto):
    """Remove quebras de linha excessivas e outros artefatos."""
    # Substitui múltiplos espaços/quebras de linha por um único espaço
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

def criar_indice_pesquisavel(texto_pdf):
    """
    Divide o texto do PDF em pedaços, cria embeddings e retorna um índice FAISS.
    """
    # 1. Dividir o texto em pedaços menores (chunks)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,  # Tamanho de cada pedaço em caracteres
        chunk_overlap=400, # Sobreposição entre pedaços para não perder contexto
        length_function=len
    )
    chunks = text_splitter.split_text(texto_pdf)

    # 2. Criar Embeddings (transformar texto em vetores)
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)

    # 3. Criar o índice FAISS a partir dos chunks e embeddings
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    return vector_store

def buscar_contexto_relevante(indice, reivindicacao, k=3):
    docs_relevantes = indice.similarity_search(reivindicacao, k=k)
    contexto = "\n---\n".join([doc.page_content for doc in docs_relevantes])
    return contexto

In [8]:
#Prompt + trata o json
def analisar_reivindicacao_com_contexto(reivindicacao, contexto):
    """
    Usa o LLM para analisar UMA ÚNICA reivindicação com base em um contexto específico.
    """
    # Prompt aprimorado com a técnica "few-shot", dando um exemplo claro do que esperamos.
    prompt = f"""<|system|> Você é um especialista em análise jurídica. Sua tarefa é avaliar se a 'REIVINDICAÇÃO' é suportada pelo 'CONTEXTO' fornecido. Responda APENAS com um único objeto JSON válido. Não adicione nenhuma explicação ou texto antes ou depois do objeto JSON. Exemplo de resposta esperada: {{ "label": "Incorreta", "evidence": "A justificativa para a incorreção, baseada estritamente no contexto." }} </s> <|user|> 'CONTEXTO': --- {contexto} --- 'REIVINDICAÇÃO': "{reivindicacao}" Gere o objeto JSON para a reivindicação acima, baseando-se estritamente no contexto fornecido.</s> <|assistant|> """

    # Prepara a entrada para o modelo
    inputs = tokenizer(prompt, return_tensors="pt", padding=False, truncation=False).to("cuda")

    # Gera a resposta do modelo
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decodifica a resposta completa
    resposta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Lógica de parsing inteligente para extrair o JSON
    try:
        # Pega apenas o que vem depois da tag do assistente
        resposta_assistente = resposta_completa.split("<|assistant|>")[1].strip()

        # Procura pelo primeiro '{' e o último '}' para extrair o bloco JSON
        match = re.search(r'\{.*\}', resposta_assistente, re.DOTALL)
        if match:
            json_str = match.group(0)
            resultado_json = json.loads(json_str)
            return resultado_json
        else:
            # Se não encontrar um JSON, registra o erro
            print(f"ERRO: Bloco JSON não encontrado na saída do modelo para a reivindicação: '{reivindicacao}'")
            print(f"Saída do modelo: {resposta_assistente}")
            return {"label": "Erro", "evidence": "Bloco JSON não encontrado na resposta do modelo."}

    except (json.JSONDecodeError, IndexError) as e:
        print(f"ERRO ao decodificar JSON para a reivindicação: '{reivindicacao}'")
        print(f"Saída do modelo: {resposta_completa}")
        return {"label": "Erro", "evidence": f"Falha ao processar a resposta do modelo: {e}"}
def extrair_reivindicacoes_regex(texto_resumo):
    """
    Extrai reivindicações unitárias de um resumo usando regex.
    Uma reivindicação = uma afirmação única.
    """
    reivindicacoes = []
    frases = re.split(r'[.;!?]\s+', texto_resumo)

    for frase in frases:
        frase = frase.strip()
        if not frase or len(frase.split()) < 3:
            continue
        subfrases = re.split(r'\s+e\s+|\s+mas\s+|\s+ou\s+', frase)
        for sub in subfrases:
            sub = sub.strip()
            if len(sub.split()) >= 3:
                reivindicacoes.append(sub)
    return reivindicacoes


def detectar_reivindicacoes_com_llm(texto_resumo, tokenizer, model):
    """
    Usa a LLM para identificar reivindicações quando regex não for suficiente.
    """
    prompt = f"""
    Identifique todas as REIVINDICAÇÕES (fatos ou afirmações independentes) no texto abaixo.
    Retorne APENAS um objeto JSON no formato:
    {{
      "reivindicacoes": ["Reivindicação 1", "Reivindicação 2", ...]
    }}

    Texto:
    {texto_resumo}
    """

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=500, pad_token_id=tokenizer.eos_token_id)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)

    try:
        match = re.search(r'\{.*\}', resposta, re.DOTALL)
        if match:
            return json.loads(match.group(0)).get("reivindicacoes", [])
    except:
        return []

    return []


def extrair_reivindicacoes(texto_resumo, tokenizer=None, model=None):
    """
    Extrai reivindicações primeiro com regex; se não encontrar nenhuma,
    tenta com a LLM (se disponível).
    """
    reivindicacoes = extrair_reivindicacoes_regex(texto_resumo)
    if not reivindicacoes and tokenizer and model:
        reivindicacoes = detectar_reivindicacoes_com_llm(texto_resumo, tokenizer, model)
    return reivindicacoes

In [9]:
#ler o arquivos
def ler_pdf(caminho_arquivo):
    """Lê o texto de um arquivo PDF."""
    texto = ""
    with open(caminho_arquivo, 'rb') as f:
        leitor = PyPDF2.PdfReader(f)
        for pagina in leitor.pages:
            texto_pagina = pagina.extract_text()
            if texto_pagina:
                texto += texto_pagina
    return texto

def ler_txt(caminho_arquivo):
    """Lê o texto de um arquivo TXT."""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return f.read()


# --- Definição dos arquivos ---
arquivos_para_analisar = [
    {
        "doc_path": "Acórdão 733 de 2025 Plenário.pdf",
        "resumo_path": "Acórdão 733-2025 resumos.txt",
        "doc_name": "Acórdão 733 de 2025 Plenário",
        "summary_id": 1
    },
    {
        "doc_path": "Acórdão 764 de 2025 Plenário.pdf",
        "resumo_path": "Acórdão 764-2025 resumos.txt",
        "doc_name": "Acórdão 764 de 2025 Plenário",
        "summary_id": 2
    }
]

analise_final = []

print("Iniciando a análise dos documentos com a nova estratégia...")

for item in arquivos_para_analisar:
    print(f"\n--- Processando: {item['doc_name']} ---")
    try:
        texto_acordao_bruto = ler_pdf(item['doc_path'])
        texto_acordao = limpar_texto(texto_acordao_bruto)
        texto_resumo = ler_txt(item['resumo_path'])

        print("Criando índice de busca para o documento...")
        indice_acordao = criar_indice_pesquisavel(texto_acordao)
        print("Índice criado com sucesso.")

        # 🚀 Extrair múltiplas reivindicações
        reivindicacoes = extrair_reivindicacoes(texto_resumo, tokenizer, model)
        print(f"Encontradas {len(reivindicacoes)} reivindicações no resumo.")

        # Analisar cada reivindicação
        for i, claim_text in enumerate(reivindicacoes):
            print(f"Analisando reivindicação {i+1}/{len(reivindicacoes)}: '{claim_text[:50]}...'")

            contexto = buscar_contexto_relevante(indice_acordao, claim_text)
            resultado_analise = analisar_reivindicacao_com_contexto(claim_text, contexto)

            if resultado_analise:
                analise_final.append({
                    "doc_name": item['doc_name'],
                    "claim_text": claim_text,
                    "label": resultado_analise.get('label', 'Erro'),
                    "evidence": resultado_analise.get('evidence', ''),
                    "summary_id": item['summary_id'],
                    "claim_id": i
                })

        print(f"Análise de '{item['doc_name']}' concluída.")

    except FileNotFoundError as e:
        print(f"ERRO: Arquivo não encontrado - {e}")
    except Exception as e:
        print(f"ERRO inesperado em {item['doc_name']}: {e}")


# ------------------- SALVAR RESULTADO -------------------
if analise_final:
    caminho_saida_json = "analise_reivindicacoes2.json"
    with open(caminho_saida_json, 'w', encoding='utf-8') as f:
        json.dump(analise_final, f, ensure_ascii=False, indent=4)

    print(f"\n--- ANÁLISE COMPLETA! ---")
    print(f"Resultado salvo em: {caminho_saida_json}")

Iniciando a análise dos documentos com a nova estratégia...

--- Processando: Acórdão 733 de 2025 Plenário ---
Criando índice de busca para o documento...


/tmp/ipython-input-1889415494.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Índice criado com sucesso.
Encontradas 13 reivindicações no resumo.
Analisando reivindicação 1/13: 'Resumo 1
O processo TC 004.980/2017-4 foi iniciado...'
Analisando reivindicação 2/13: 'A principal conclusão do TCU foi o reconhecimento ...'
Analisando reivindicação 3/13: 'O acórdão também determinou que o banco restituíss...'
Analisando reivindicação 4/13: 'Resumo 2
A representação TC 004.980/2017-4, aprese...'
Analisando reivindicação 5/13: 'Segundo o TCU, embora o banco utilize recursos púb...'
Analisando reivindicação 6/13: 'FMM, isso não o caracteriza como dependente da Uni...'
Analisando reivindicação 7/13: 'No entanto, o tribunal decidiu que todos os benefí...'
Analisando reivindicação 8/13: 'PLR, deveriam ser imediatamente cortados, mesmo os...'
Analisando reivindicação 9/13: 'Resumo 3
O Acórdão 733/2025 trata do pedido do TCU...'
Analisando reivindicação 10/13: 'A decisão final determinou o reconhecimento da dep...'
Analisando reivindicação 11/13: 'o bloqueio do pagamento de b